<a href="https://colab.research.google.com/github/rishabkolan/Projects/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **UFC fight duration prediction**

# 1. Data Preprocessing

In [269]:
#Importing the Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive

In [270]:
#Creating the dataframe

drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/Classroom/large_dataset.csv'
df = pd.read_csv(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [271]:
df.head(5)

,event_name,r_fighter,b_fighter,winner,weight_class,is_title_bout,gender,method,finish_round,total_rounds,...,weight_diff,reach_diff,SLpM_total_diff,SApM_total_diff,sig_str_acc_total_diff,td_acc_total_diff,str_def_total_diff,td_def_total_diff,sub_avg_diff,td_avg_diff
0,UFC Fight Night: Ribas vs. Namajunas,Amanda Ribas,Rose Namajunas,Blue,Women's Flyweight,0,Women,Decision - Unanimous,5,5.0,...,0.00,2.54,0.94,-0.11,-0.01,0.04,-0.02,0.26,0.2,0.69
1,UFC Fight Night: Ribas vs. Namajunas,Karl Williams,Justin Tafa,Red,Heavyweight,0,Men,Decision - Unanimous,3,3.0,...,-13.16,12.70,-1.22,-3.32,-0.02,0.50,0.13,0.50,0.2,4.75
2,UFC Fight Night: Ribas vs. Namajunas,Edmen Shahbazyan,AJ Dobson,Red,Middleweight,0,Men,KO/TKO,1,3.0,...,0.00,-2.54,-0.69,-1.22,0.06,-0.37,-0.01,-0.02,0.3,0.57
3,UFC Fight Night: Ribas vs. Namajunas,Payton Talbott,Cameron Saaiman,Red,Bantamweight,0,Men,KO/TKO,2,3.0,...,0.00,7.62,2.73,-0.60,0.08,-0.28,0.00,0.43,-0.2,-0.91
4,UFC Fight Night: Ribas vs. Namajunas,Billy Quarantillo,Youssef Zalal,Blue,Featherweight,0,Men,Submission,2,3.0,...,0.00,-5.08,4.48,3.84,0.07,-0.11,-0.22,0.01,-0.2,-1.04


In [272]:
df.columns

Index(['event_name', 'r_fighter', 'b_fighter', 'winner', 'weight_class',
       'is_title_bout', 'gender', 'method', 'finish_round', 'total_rounds',
       'time_sec', 'referee', 'r_kd', 'r_sig_str', 'r_sig_str_att',
       'r_sig_str_acc', 'r_str', 'r_str_att', 'r_str_acc', 'r_td', 'r_td_att',
       'r_td_acc', 'r_sub_att', 'r_rev', 'r_ctrl_sec', 'r_wins_total',
       'r_losses_total', 'r_age', 'r_height', 'r_weight', 'r_reach',
       'r_stance', 'r_SLpM_total', 'r_SApM_total', 'r_sig_str_acc_total',
       'r_td_acc_total', 'r_str_def_total', 'r_td_def_total', 'r_sub_avg',
       'r_td_avg', 'b_kd', 'b_sig_str', 'b_sig_str_att', 'b_sig_str_acc',
       'b_str', 'b_str_att', 'b_str_acc', 'b_td', 'b_td_att', 'b_td_acc',
       'b_sub_att', 'b_rev', 'b_ctrl_sec', 'b_wins_total', 'b_losses_total',
       'b_age', 'b_height', 'b_weight', 'b_reach', 'b_stance', 'b_SLpM_total',
       'b_SApM_total', 'b_sig_str_acc_total', 'b_td_acc_total',
       'b_str_def_total', 'b_td_def_total', 'b_

In [273]:
#Found null values after running the reverse order of df
df = df.dropna()

In [274]:
df[['finish_round', 'total_rounds', 'time_sec']]

,finish_round,total_rounds,time_sec
0,5,5.0,300
1,3,3.0,300
2,1,3.0,273
3,2,3.0,21
4,2,3.0,110
...,...,...,...
7184,3,3.0,300
7207,5,5.0,300
7250,1,1.0,720
7263,1,1.0,44


In [275]:
#Finding the target values
df['target'] = (df['finish_round'] -1)*300 + df['time_sec']

In [276]:
# Preparing df to be in order to prepare the train, valid, and test datasets
df = df.iloc[::-1]

In [277]:
df = df.reset_index()
df = df.drop(columns = ['index'], axis = 1)


In [278]:
df.columns[df.columns.get_loc('kd_diff'):]

Index(['kd_diff', 'sig_str_diff', 'sig_str_att_diff', 'sig_str_acc_diff',
       'str_diff', 'str_att_diff', 'str_acc_diff', 'td_diff', 'td_att_diff',
       'td_acc_diff', 'sub_att_diff', 'rev_diff', 'ctrl_sec_diff',
       'wins_total_diff', 'losses_total_diff', 'age_diff', 'height_diff',
       'weight_diff', 'reach_diff', 'SLpM_total_diff', 'SApM_total_diff',
       'sig_str_acc_total_diff', 'td_acc_total_diff', 'str_def_total_diff',
       'td_def_total_diff', 'sub_avg_diff', 'td_avg_diff', 'target'],
      dtype='object')

In [279]:
df['r_ctrl_sec_total'] = df['r_ctrl_sec'][:]
df['b_ctrl_sec_total'] = df['b_ctrl_sec'][:]
df['r_kd_total'] = df['r_kd'][:]
df['b_kd_total'] = df['b_kd'][:]

In [280]:
df['r_td_def'] = df['b_td_att'] - df['b_td']
df['b_td_def'] = df['r_td_att'] - df['r_td']
df['r_str_acc_total'] = df['r_str_acc'][:]
df['b_str_acc_total'] = df['b_str_acc'][:]
df['r_sub_att_total'] = df['r_sub_att'][:]
df['b_sub_att_total'] = df['b_sub_att'][:]

In [309]:
#Found out the stats of each fighter is constant in all their fights which isn't true. Also found out that the avg number of knockdowns was missing which is a key component.

charles_red = df[df['r_fighter'] == 'Charles Oliveira']
charles_blue = df[df['b_fighter'] == 'Charles Oliveira']
charles = pd.concat([charles_red, charles_blue]).sort_index()




(np.int64(1), np.int64(1))

In [305]:
indices = charles.index.sort_values(ascending = True)
indices

Index([ 745,  779,  836,  982, 1205, 1279, 1523, 1733, 1875, 2066, 2238, 2360,
       2470, 2738, 2810, 3001, 3298, 3519, 3660, 3780, 3818, 3987, 4260, 4389,
       4722, 4922, 5217, 5414, 5651, 5960],
      dtype='int64')

In [285]:
red_features = ['r_kd', 'r_sig_str_acc', 'r_str_acc', 'r_td_acc','r_sub_att', 'r_ctrl_sec', 'r_td_def' ]
blue_features = ['b_kd', 'b_sig_str_acc', 'b_str_acc', 'b_td_acc','b_sub_att', 'b_ctrl_sec', 'b_td_def' ]

In [286]:
charles[red_features[0] + '_total'].loc[5960]

np.int64(1)

In [303]:
feature_length = len(red_features)
for n in np.arange(feature_length):
  count = 0
  average = 0
  sum = 0
  for index in indices:
    count += 1
    if charles['r_fighter'].loc[index] == 'Charles Oliveira':

      try:
        average = sum / (count - 1)
      except ZeroDivisionError:
        average = 0
      sum += charles.loc[index, red_features[n]]
      if count == 1:
        continue
      else:
        charles.loc[index, red_features[n] + '_total'] = average
    else:

      try:
        average = sum / (count - 1)
      except ZeroDivisionError:
        average = 0
      sum += charles.loc[index, blue_features[n]]
      if count == 1:
        continue
      else:
        charles.loc[index, blue_features[n] + '_total'] = average



In [304]:
charles

,event_name,r_fighter,b_fighter,winner,weight_class,is_title_bout,gender,method,finish_round,total_rounds,...,r_ctrl_sec_total,b_ctrl_sec_total,r_kd_total,b_kd_total,r_td_def,b_td_def,r_str_acc_total,b_str_acc_total,r_sub_att_total,b_sub_att_total
745,UFC Live: Jones vs Matyushenko,Darren Elkins,Charles Oliveira,Blue,Lightweight,0,Men,Submission,1,3.0,...,27.000000,0.000000,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0.000000
779,UFC Fight Night: Marquardt vs Palhares,Efrain Escudero,Charles Oliveira,Blue,Lightweight,0,Men,Submission,3,3.0,...,167.000000,0.000000,0.000000,0.000000,7,1,0.290000,0.750000,0.000000,2.000000
836,UFC 124: St-Pierre vs Koscheck 2,Jim Miller,Charles Oliveira,Red,Lightweight,0,Men,Submission,1,3.0,...,91.000000,32.000000,0.000000,0.000000,1,0,0.550000,0.630000,2.000000,1.500000
982,UFC Live: Hardy vs Lytle,Donald Cerrone,Charles Oliveira,Red,Lightweight,0,Men,KO/TKO,1,3.0,...,12.000000,21.333333,0.000000,0.000000,0,0,0.510000,0.643333,0.000000,1.000000
1205,The Ultimate Fighter: Live Finale,Jonathan Brookins,Charles Oliveira,Blue,Featherweight,0,Men,Submission,2,3.0,...,67.000000,16.000000,0.000000,0.000000,1,1,0.360000,0.567500,1.000000,0.750000
1279,UFC 152: Jones vs Belfort,Cub Swanson,Charles Oliveira,Red,Featherweight,0,Men,KO/TKO,1,3.0,...,2.000000,24.600000,1.000000,0.000000,1,0,0.430000,0.572000,0.000000,0.800000
1523,UFC 162: Silva vs Weidman,Frankie Edgar,Charles Oliveira,Red,Featherweight,0,Men,Decision - Unanimous,3,3.0,...,303.000000,28.166667,0.000000,0.000000,4,3,0.630000,0.551667,0.000000,0.666667
1733,UFC Fight Night: Machida vs Mousasi,Charles Oliveira,Andy Ogle,Red,Featherweight,0,Men,Submission,3,3.0,...,24.285714,269.000000,0.000000,0.000000,1,5,0.530000,0.550000,0.714286,0.000000
1875,UFC Fight Night: Te Huna vs Marquardt,Hatsu Hioki,Charles Oliveira,Blue,Featherweight,0,Men,Submission,2,3.0,...,357.000000,67.375000,0.000000,0.000000,2,1,0.610000,0.553750,0.000000,0.875000
2066,The Ultimate Fighter: A Champion Will Be Crown...,Jeremy Stephens,Charles Oliveira,Blue,Featherweight,0,Men,Decision - Unanimous,3,3.0,...,379.000000,75.777778,0.000000,0.000000,5,0,0.620000,0.562222,0.000000,1.000000


In [310]:
def accurate_stats(dataframe, r_features, b_features, fighters):

  assert len(r_features) == len(b_features)

  for fighter in fighters:
    fighter_red, fighter_blue = dataframe[dataframe['r_fighter'] == fighter], dataframe[dataframe['b_fighter'] == fighter]
    indices = pd.concat([fighter_red, fighter_blue]).sort_index().index
    feature_length = len(r_features)

    for n in np.arange(feature_length):
        count = 0
        average = 0
        sum = 0
        for index in indices:
          count += 1
          if dataframe['r_fighter'].loc[index] == fighter:

            try:
              average = sum / (count - 1)
            except ZeroDivisionError:
              average = 0
            sum += dataframe.loc[index, r_features[n]]
            if count == 1:
              continue
            else:
              dataframe.loc[index, r_features[n] + '_total'] = average
          else:

            try:
              average = sum / (count - 1)
            except ZeroDivisionError:
              average = 0
            sum += dataframe.loc[index, b_features[n]]
            if count == 1:
              continue
            else:
              dataframe.loc[index, b_features[n] + '_total'] = average

  return dataframe






In [321]:
fighters_array = np.unique(np.concatenate([np.unique(df['r_fighter']), np.unique(df['b_fighter'])]))
len(fighters_array)

1812

In [322]:
accurate_stats(df, red_features, blue_features, fighters_array)

/tmp/ipython-input-2886355647.py:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '116.33333333333333' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dataframe.loc[index, b_features[n] + '_total'] = average
/tmp/ipython-input-2886355647.py:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dataframe.loc[index, b_features[n] + '_total'] = average
/tmp/ipython-input-2886355647.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.3333333333333333' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dataframe.loc[index, r_features[n] + '_total'] = average
/tmp/ipython-input-

,event_name,r_fighter,b_fighter,winner,weight_class,is_title_bout,gender,method,finish_round,total_rounds,...,r_ctrl_sec_total,b_ctrl_sec_total,r_kd_total,b_kd_total,r_td_def,b_td_def,r_str_acc_total,b_str_acc_total,r_sub_att_total,b_sub_att_total
0,UFC 15: Collision Course,Randy Couture,Vitor Belfort,Red,Heavyweight,0,Men,KO/TKO,1,1.0,...,0.000000,0.000000,0.000000,0.000000,1,0,0.880000,0.460000,1.000000,0.000000
1,UFC - Ultimate Brazil,Vitor Belfort,Wanderlei Silva,Red,Middleweight,0,Men,KO/TKO,1,1.0,...,0.000000,0.000000,0.000000,0.000000,0,0,0.460000,0.500000,0.000000,0.000000
2,UFC 19: Ultimate Young Guns,Jeremy Horn,Chuck Liddell,Red,Middleweight,0,Men,Submission,1,1.0,...,0.000000,0.000000,0.000000,0.000000,0,4,0.810000,0.830000,3.000000,0.000000
3,UFC 25: Ultimate Japan 3,Tito Ortiz,Wanderlei Silva,Red,UFC Light Heavyweight Title,1,Men,Decision - Unanimous,5,5.0,...,1153.000000,0.000000,1.000000,0.000000,1,2,0.800000,0.500000,0.000000,0.000000
4,UFC 29: Defense of the Belts,Chuck Liddell,Jeff Monson,Red,Middleweight,0,Men,Decision - Unanimous,3,3.0,...,0.000000,63.000000,0.000000,0.000000,16,0,0.830000,0.560000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6356,UFC Fight Night: Ribas vs. Namajunas,Billy Quarantillo,Youssef Zalal,Blue,Featherweight,0,Men,Submission,2,3.0,...,204.555556,316.000000,0.222222,0.166667,0,0,0.643333,0.573333,1.000000,1.166667
6357,UFC Fight Night: Ribas vs. Namajunas,Payton Talbott,Cameron Saaiman,Red,Bantamweight,0,Men,KO/TKO,2,3.0,...,205.000000,204.750000,0.000000,0.250000,3,1,0.640000,0.575000,1.000000,0.500000
6358,UFC Fight Night: Ribas vs. Namajunas,Edmen Shahbazyan,AJ Dobson,Red,Middleweight,0,Men,KO/TKO,1,3.0,...,112.555556,116.333333,0.333333,0.000000,0,2,0.546667,0.520000,0.333333,0.000000
6359,UFC Fight Night: Ribas vs. Namajunas,Karl Williams,Justin Tafa,Red,Heavyweight,0,Men,Decision - Unanimous,3,3.0,...,384.000000,37.428571,0.500000,0.571429,0,5,0.580000,0.635714,0.000000,0.000000
